### Gate and Noise Parameters

We define the basic physical parameters used in simulating a single-qubit noisy quantum gate:

- `theta`: angle of rotation on the Bloch sphere (gate strength),
- `phi`: phase angle determining the rotation axis,
- `p`: depolarizing error probability,
- `T1`, `T2`: relaxation and dephasing times (ns),
- `tg`: gate time (duration of applied control), typically $ 35 \text{ ns} $.

The noise strength parameters are derived as:
$$
\epsilon_d = \sqrt{\frac{p}{4}}, \quad
\epsilon_1 = \sqrt{\frac{t_g}{T_1}}, \quad
\epsilon_\phi = \sqrt{\frac{1}{2} \left( \epsilon_2^2 - \frac{\epsilon_1^2}{2} \right)}
$$


In [34]:
import numpy as np
from numpy import exp, sqrt, array
from scipy.linalg import expm
import scipy
from numpy.random import multivariate_normal
from qiskit import QuantumCircuit, transpile, assemble
from qiskit.quantum_info import partial_trace
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram, plot_bloch_multivector
import matplotlib.pyplot as plt
from qiskit.quantum_info import Operator
from scipy.linalg import expm
from qiskit.circuit.library import UnitaryGate

In [35]:

def parameters(pp, pT1, pT2, ptg):
    p = pp                  # depolarizing probability
    T1 = pT1                 # relaxation time (seconds)
    T2 = pT2                 # dephasing time (seconds)
    tg = ptg                 # gate time (seconds)

    # Derived noise amplitudes
    ed = sqrt(p / 4)
    e1 = sqrt(tg / T1) if T1 else 0
    e2 = sqrt(tg / T2) if T2 else 0
    ep = sqrt(0.5 * (e2**2 - e1**2 / 2)) if T2 else 0
    return ed, e1, e2, ep

### 1. Unitary Contribution

The ideal gate corresponds to a rotation of angle $ \theta $ around an axis in the XYZ-plane determined by $ \phi $. The corresponding unitary is:
$$
U(\theta, \psi, \phi)= \begin{pmatrix}
\cos\left( \frac{\theta}{2} \right)-i\sin\left( \frac{\theta}{2} \right)\cos(\psi) & -i \sin\left( \frac{\theta}{2} \right)\sin(\psi) e^{-i\phi} \\
-i \sin\left( \frac{\theta}{2} \right) \sin(\psi) e^{i\phi} & \cos\left( \frac{\theta}{2} \right)+i\sin\left( \frac{\theta}{2} \right)\cos(\psi) 
\end{pmatrix}
$$


In [4]:
def make_U(theta,phi):
    U = array([
            [np.cos(theta/2), -1j * np.sin(theta/2) * exp(-1j * phi)],
            [-1j * np.sin(theta/2) * exp(1j * phi), np.cos(theta/2)]
        ])
    return U


In [36]:

def make_U(theta, phi, psi):
    # Rotation axis: n = (sinψ cosφ, sinψ sinφ, cosψ)
    nx = np.sin(psi) * np.cos(phi)
    ny = np.sin(psi) * np.sin(phi)
    nz = np.cos(psi)

    # Build unitary via Rodrigues' rotation formula
    I = np.eye(2)
    X = np.array([[0, 1], [1, 0]])
    Y = np.array([[0, -1j], [1j, 0]])
    Z = np.array([[1, 0], [0, -1]])

    n_dot_sigma = nx * X + ny * Y + nz * Z
    return scipy.linalg.expm(-1j * theta / 2 * n_dot_sigma)


### 2. Depolarization Noise Decomposition

In the paper ["Noisy Gates for Simulating Quantum Computers" (Phys. Rev. Research 5, 043210)], depolarizing noise is modeled as a stochastic process applied during the gate operation. The depolarizing channel for a single qubit is formally given by:

$$
\mathcal{E}(\rho) = (1 - p) \rho + \frac{p}{3}(X \rho X + Y \rho Y + Z \rho Z),
$$

where $ p $ is the depolarizing probability. Instead of applying this as a discrete channel, the authors model this noise continuously during the gate using time-dependent Lindblad evolution with jump operators $ L_k \propto \sigma_k $ (Pauli matrices).

In the **interaction picture**, the Lindblad operators become time-dependent and are denoted:

$$
L_k(s) = U_s^\dagger \sigma_k U_s,
$$

where $ U_s $ is the unitary evolution operator generated by the applied gate. The stochastic evolution of the system is then unraveled using Itô calculus and expanded to second order in the noise strength.

Depolarizing noise is modeled as a stochastic perturbation along the Pauli $ X, Y, Z $ directions, added to the ideal unitary gate evolution. In the paper, this is expressed using a stochastic differential equation (SDE) unravelling of the Lindblad master equation. Specifically, the depolarizing channel is expanded into time-dependent Lindblad operators:

$$
L_X(s) = \sqrt{\frac{p}{4t_g}} \, X(s), \quad
L_Y(s) = \sqrt{\frac{p}{4t_g}} \, Y(s), \quad
L_Z(s) = \sqrt{\frac{p}{4t_g}} \, Z(s),
$$

and their contributions are captured in the stochastic term:

$$
\Xi := i\epsilon \sum_{k=1}^{N^2 - 1} \int_0^1 dW_{k,s} \, L_{k,s},
\tag{14}
$$

where $ \epsilon = \sqrt{p/4} $ and $ W_{k,s} $ are independent Wiener processes (Brownian motions). This Hermitian operator $ \Xi $ represents the stochastic component of the noisy gate evolution.

Mathematically, each stochastic term of the form
$$
\int_0^1 dW_{k,s} \, L_k(s)
$$
is a **zero-mean Gaussian random variable**, due to the Itô isometry. Its variance is given by
$$
\mathbb{E}\left[ \left( \int_0^1 dW_{k,s} \, f_k(s) \right)^2 \right] = \int_0^1 f_k^2(s) \, ds,
$$
where $ f_k(s) $ is the scalar function accompanying the stochastic operator $ L_k(s) $ under the integral. 

To simulate this correctly, the integral is not evaluated directly, but instead **sampled from a multivariate normal distribution** with a covariance matrix derived from these integrals:
$$
\text{Cov}[\mathcal{I}_i, \mathcal{I}_j] = \int_0^1 f_i(s) f_j(s) \, ds.
$$
This guarantees that the stochastic noise term $ \Xi $ matches the correct second-order statistics of the underlying continuous-time evolution. The Hermitian matrices $ \texttt{Idx}, \texttt{Idy}, \texttt{Idz} $ are then constructed from the sampled components of these integrals.


To simulate this noise numerically, the stochastic increments are sampled from **multivariate normal distributions** whose covariance matrices are derived from the second-order moments of the corresponding Itô integrals (as discussed around Eq. (14) in the paper). For each axis (X, Y, Z), the Itô expansion includes:

- Two stochastic integrals  $ \mathcal{I}_1, \mathcal{I}_2 $: integrals of time-dependent coefficients,
- $ W $: the Wiener process (Brownian motion),
- Covariance structure: derived from integrals over functions like $ \sin^2(\theta/2) $, $ \cos^2(\theta/2) $, etc.

The matrices $ \texttt{Idx}, \texttt{Idy}, \texttt{Idz} $ represent the effective Hermitian noise operators for each component, and are sampled from multivariate normal distributions with these covariances.

This continuous-time modeling of depolarizing noise allows it to interact with the gate dynamics in a physically realistic way, and permits perturbative expansion consistent with open quantum systems theory.


### 2.1 Depolarization Noise — X Component

Depolarization is modeled via stochastic processes along different axes. For the **X-component**, the following integrals are used to construct the Itô covariance matrix:

$$
\begin{aligned}
\mathbb{V}[\mathcal{I}_1^{(X)}] &= \frac{2\theta - \sin(2\theta)}{4\theta}, \\
\mathbb{V}[\mathcal{I}_2^{(X)}] &= \frac{6\theta - 8\sin(\theta) + \sin(2\theta)}{16\theta}, \\
\text{Cov}[\mathcal{I}_1, \mathcal{I}_2] &= \frac{\sin^4(\theta/2)}{\theta}, \\
\text{Cov}[\mathcal{I}_1, W] &= \frac{1 - \cos(\theta)}{\theta}, \\
\text{Cov}[\mathcal{I}_2, W] &= \frac{\theta - \sin(\theta)}{2\theta}.
\end{aligned}
$$

We sample from this distribution and build the contribution $ \texttt{Idx} $ to the noisy evolution.


In [37]:
def depolarization_noise_x(theta,phi,ed):
    # Variances and covariances for depolarization Itô processes (X axis)
    Vdx_1 = (2*theta - np.sin(2*theta)) / (4*theta)
    Vdx_2 = (6*theta - 8*np.sin(theta) + np.sin(2*theta)) / (16*theta)
    Covdx_12 = (np.sin(theta/2))**4 / theta
    Covdx_1Wdx = (1 - np.cos(theta)) / theta
    Covdx_2Wdx = (theta - np.sin(theta)) / (2 * theta)

    meand_x = array([0, 0, 0])
    covd_x = array([
        [Vdx_1, Covdx_12, Covdx_1Wdx],
        [Covdx_12, Vdx_2, Covdx_2Wdx],
        [Covdx_1Wdx, Covdx_2Wdx, 1]
    ])

    sample_dx = multivariate_normal(meand_x, covd_x, 1)
    Idx1, Idx2, Wdx = sample_dx[0]

    Idx = ed * array([
        [np.sin(phi) * Idx1, Wdx + (exp(-2j*phi) - 1) * Idx2],
        [Wdx + (exp(2j*phi) - 1) * Idx2, -np.sin(phi) * Idx1]
    ])
    return Vdx_1, Vdx_2, Covdx_12, Covdx_1Wdx, Covdx_2Wdx, Idx


### 2.2 Depolarization Noise — Y Component

The Itô integrals for the **Y-component** use the same formulas as X, due to symmetry, but contribute different operator structure:

$$
\begin{aligned}
\mathbb{V}[\mathcal{I}_1^{(Y)}] &= \mathbb{V}[\mathcal{I}_1^{(X)}], \\
\text{and so on...}
\end{aligned}
$$

These contribute to a matrix $ \texttt{Idy} $ as:
$$
\texttt{Idy} = \epsilon_d \begin{pmatrix}
-\cos(\phi) \cdot I_1 & -i W + i(e^{-2i\phi} + 1) I_2 \\
i W - i(e^{2i\phi} + 1) I_2 & \cos(\phi) \cdot I_1
\end{pmatrix}
$$


In [38]:
def depolarization_noise_y(phi,ed, Vdx_1, Vdx_2, Covdx_12, Covdx_1Wdx, Covdx_2Wdx):
    Vdy_1 = Vdx_1
    Vdy_2 = Vdx_2
    Covdy_12 = Covdx_12
    Covdy_1Wdy = Covdx_1Wdx
    Covdy_2Wdy = Covdx_2Wdx

    meand_y = array([0, 0, 0])
    covd_y = array([
        [Vdy_1, Covdy_12, Covdy_1Wdy],
        [Covdy_12, Vdy_2, Covdy_2Wdy],
        [Covdy_1Wdy, Covdy_2Wdy, 1]
    ])

    sample_dy = multivariate_normal(meand_y, covd_y, 1)
    Idy1, Idy2, Wdy = sample_dy[0]

    Idy = ed * array([
        [-np.cos(phi) * Idy1, -1j * Wdy + 1j * (exp(-2j*phi) + 1) * Idy2],
        [1j * Wdy - 1j * (exp(2j*phi) + 1) * Idy2, np.cos(phi) * Idy1]
    ])
    return Idy



### 2.3 Depolarization Noise — Z Component

The Z-component includes only two Itô integrals with:

$$
\begin{aligned}
\mathbb{V}[\mathcal{I}_1^{(Z)}] &= \frac{2\theta + \sin(2\theta)}{4\theta}, \\
\mathbb{V}[\mathcal{I}_2^{(Z)}] &= \frac{2\theta - \sin(2\theta)}{4\theta}, \\
\text{Cov}[\mathcal{I}_1, \mathcal{I}_2] &= \frac{\sin^2(\theta)}{2\theta}.
\end{aligned}
$$

These contribute to a Hermitian correction term on the Z axis:


In [39]:
def depolarization_noise_z(theta,phi,ed):
    Vdz_1 = (2*theta + np.sin(2*theta)) / (4*theta)
    Vdz_2 = (2*theta - np.sin(2*theta)) / (4*theta)
    Covdz_12 = (np.sin(theta))**2 / (2*theta)

    meand_z = array([0, 0])
    covd_z = array([
        [Vdz_1, Covdz_12],
        [Covdz_12, Vdz_2]
    ])

    sample_dz = multivariate_normal(meand_z, covd_z, 1)
    Idz1, Idz2 = sample_dz[0]

    Idz = ed * array([
        [Idz1, -1j * exp(-1j*phi) * Idz2],
        [1j * exp(1j*phi) * Idz2, -Idz1]
    ])
    return Idz


### 3 Relaxation Contributions 
### 3.1 — $ \sigma_- $ Channel

This component models **amplitude damping** via a Lindblad operator:
$$
L_{\downarrow}(s) = \sqrt{\frac{1}{T_1}} \, \sigma_-(s),
$$
where $ \sigma_-(s) $ is the lowering operator in the interaction picture.

The effective Hamiltonian governing IBM’s native gate implementation is defined in Eq. (17) of the paper:
$$
H(\theta, \phi) = \frac{\theta \hbar}{2} R_{xy}(\phi), \quad \text{with} \quad R_{xy}(\phi) = \cos(\phi) X + \sin(\phi) Y,
$$
which generates a rotation of angle $ \theta $ about an axis in the $ XY $-plane, determined by $ \phi $.

Relaxation noise is included during this gate using a stochastic unravelling of the Lindblad master equation. Following the derivation around Eq. (17), the Itô integrals contributing to amplitude damping are approximated by:

$$
\begin{aligned}
\mathbb{V}[\mathcal{I}_1] &= \frac{2\theta - \sin(2\theta)}{4\theta}, \\
\mathbb{V}[\mathcal{I}_2] &= \frac{6\theta - 8\sin(\theta) + \sin(2\theta)}{16\theta}, \\
\text{Cov}[\mathcal{I}_1, \mathcal{I}_2] &= \frac{\sin^4(\theta/2)}{\theta}, \\
\text{Cov}[\mathcal{I}_1, W] &= \frac{1 - \cos(\theta)}{\theta}, \\
\text{Cov}[\mathcal{I}_2, W] &= \frac{\theta - \sin(\theta)}{2\theta}.
\end{aligned}
$$

These are used to construct a 3×3 multivariate Gaussian from which random samples are drawn and scaled by the amplitude damping factor:
$$
\epsilon_1 = \sqrt{\frac{t_g}{T_1}}.
$$

The resulting Hermitian matrix $ \texttt{Ir} $ contributes to the total noise applied during the gate.


In [40]:
def stochastic_amplitude_damping(theta, phi, e1):
    # 3) RELAXATION CONTRIBUTIONS — sigma_minus channel (Amplitude Damping)
    Vr1 = (2*theta - np.sin(2*theta)) / (4*theta)
    Vr2 = (6*theta - 8*np.sin(theta) + np.sin(2*theta)) / (16*theta)
    Covr12 = (np.sin(theta / 2))**4 / theta
    Covr1Wr = (1 - np.cos(theta)) / theta
    Covr2Wr = (theta - np.sin(theta)) / (2 * theta)

    meanr = array([0, 0, 0])
    covr = array([
        [Vr1, Covr12, Covr1Wr],
        [Covr12, Vr2, Covr2Wr],
        [Covr1Wr, Covr2Wr, 1]
    ])

    sample_r = multivariate_normal(meanr, covr, 1)
    Ir1, Ir2, Wr = sample_r[0]

    Ir = e1 * array([
        [-1j/2 * exp(1j * phi) * Ir1, Wr - Ir2],
        [exp(2j * phi) * Ir2, 1j/2 * exp(1j * phi) * Ir1]
    ])
    return Ir


### 3.2 Deterministic Relaxation Contribution

In addition to the stochastic contributions from amplitude damping, there is a deterministic component arising from the average (non-stochastic) part of the Lindblad evolution.

This is captured by the term:
$$
\Lambda := -\frac{\epsilon_1^2}{2} \int_0^1 \left( L_{\downarrow,s}^\dagger L_{\downarrow,s} \right) ds,
$$
which leads to a Hermitian drift term added to the noisy gate.

The corresponding integrals are:
$$
\begin{aligned}
\text{det}_1 &= \frac{\theta - \sin(\theta)}{2\theta}, \\
\text{det}_2 &= \frac{1 - \cos(\theta)}{\theta}, \\
\text{det}_3 &= \frac{\theta + \sin(\theta)}{2\theta},
\end{aligned}
$$

and they form the matrix:
$$
\texttt{deterministic} = -\frac{\epsilon_1^2}{2}
\begin{pmatrix}
\text{det}_1 & \frac{i}{2} e^{-i\phi} \text{det}_2 \\
-\frac{i}{2} e^{i\phi} \text{det}_2 & \text{det}_3
\end{pmatrix}
$$


In [41]:
def deterministic_amplitude_damping(theta, phi,e1):
    # 3 continued) Deterministic relaxation contribution
    det1 = (theta - np.sin(theta)) / (2 * theta)
    det2 = (1 - np.cos(theta)) / theta
    det3 = (theta + np.sin(theta)) / (2 * theta)

    deterministic = -e1**2 / 2 * array([
        [det1, 1j/2 * exp(-1j * phi) * det2],
        [-1j/2 * exp(1j * phi) * det2, det3]
    ])
    return deterministic

### 3.3 Variances and covariances for relaxation Itô processes depending on Z(t)
**Dephasing Noise — $ \sigma_z $ Channel**
Pure dephasing (phase damping) is modeled via a Lindblad operator:
$$
L_\phi(s) = \sqrt{\frac{1}{T_2}} \, \sigma_z(s),
$$
where $ \sigma_z(s) $ is the Pauli-Z operator in the interaction picture.

As with other stochastic terms, we expand this in second-order Itô integrals. The statistical quantities used to build the covariance matrix are:
$$
\begin{aligned}
\mathbb{V}[\mathcal{I}_1] &= \frac{2\theta + \sin(2\theta)}{4\theta}, \\
\mathbb{V}[\mathcal{I}_2] &= \frac{2\theta - \sin(2\theta)}{4\theta}, \\
\text{Cov}[\mathcal{I}_1, \mathcal{I}_2] &= \frac{\sin^2(\theta)}{2\theta}.
\end{aligned}
$$

This results in a 2×2 Hermitian contribution matrix $ \texttt{Ip} $, scaled by the dephasing amplitude:
$$
\epsilon_\phi = \sqrt{\frac{1}{2} \left( \frac{t_g}{T_2} - \frac{t_g}{2T_1} \right)}.
$$

In [42]:
def dephasing_noise_z(theta, phi, ep):
    # 3 continued) Dephasing Noise: sigma_z (pure dephasing) channel
    Vp_1 = (2*theta + np.sin(2*theta)) / (4*theta)
    Vp_2 = (2*theta - np.sin(2*theta)) / (4*theta)
    Covp_12 = (np.sin(theta))**2 / (2*theta)

    meanp = array([0, 0])
    covp = array([
        [Vp_1, Covp_12],
        [Covp_12, Vp_2]
    ])

    sample_p = multivariate_normal(meanp, covp, 1)
    Ip1, Ip2 = sample_p[0]

    Ip = ep * array([
        [Ip1, -1j * exp(-1j * phi) * Ip2],
        [1j * exp(1j * phi) * Ip2, -Ip1]
    ])
    return Ip

### 3.4 Final Noisy Gate Construction

After computing the unitary evolution $ U $, the deterministic drift term (relaxation), and five Hermitian noise contributions from stochastic depolarization and relaxation:

- $ \texttt{deterministic} $: the drift term from amplitude damping (Sec. 3),
- $ \texttt{Idx}, \texttt{Idy}, \texttt{Idz} $: stochastic Hermitian terms from depolarization noise (Sec. 2),
- $ \texttt{Ir}, \texttt{Ip} $: stochastic Hermitian terms from amplitude damping and dephasing (Sec. 3),

we form the total noisy gate using a second-order Trotter approximation:
$$
\texttt{result} = U \cdot e^{\texttt{deterministic}} \cdot e^{i (\texttt{Idx} + \texttt{Idy} + \texttt{Idz} + \texttt{Ir} + \texttt{Ip})}.
$$

This yields a single-qubit gate matrix with realistic stochastic and dissipative noise, matching the evolution model described in Eq. (13)–(14) of the paper.


In [43]:
# 3.4 Combine all terms to form the final noisy gate
def gate_construction(para_theta, para_phi, para_psi, para_p, para_T1, para_T2, para_tg):
    theta = para_theta
    phi = para_phi
    psi = para_psi
    ed, e1, e2, ep = parameters(para_p, para_T1, para_T2, para_tg)

    U = make_U(theta,phi,psi)
    deterministic = deterministic_amplitude_damping(theta, phi, e1)
    Vdx_1, Vdx_2, Covdx_12, Covdx_1Wdx, Covdx_2Wdx, Idx = depolarization_noise_x(theta,phi,ed)
    Idy = depolarization_noise_y(phi,ed, Vdx_1, Vdx_2, Covdx_12, Covdx_1Wdx, Covdx_2Wdx)
    Idz = depolarization_noise_z(theta,phi,ed)
    Ir = stochastic_amplitude_damping(theta, phi, e1)
    Ip = dephasing_noise_z(theta, phi, ep)

    result = U @ expm(deterministic) @ expm(1j * (Idx + Idy + Idz + Ir + Ip))
    return result

###  X and Y gate implementation

The ideal gate corresponds to a rotation of angle $ \theta $ around an axis in the XY-plane determined by $ \phi $. The corresponding unitary is:
$$
U = \begin{pmatrix}
\cos\left( \frac{\theta}{2} \right)-i\sin\left( \frac{\theta}{2} \right)\cos(\psi) & -i \sin\left( \frac{\theta}{2} \right)\sin(\psi) e^{-i\phi} \\
-i \sin\left( \frac{\theta}{2} \right) \sin(\psi) e^{i\phi} & \cos\left( \frac{\theta}{2} \right)+i\sin\left( \frac{\theta}{2} \right)\cos(\psi) 
\end{pmatrix}
$$

X Gate: 
$$\theta = \pi, \phi = 0, \psi = \frac{\pi}{2}, $$

Y Gate: 
$$\theta = \pi, \phi = \pi/2, \psi = \pi/2 $$

Z Gate: 
$$\theta = \pi, \phi = \pi, \psi = don't matter $$

I Gate: 
$$\theta = 2\pi, \phi = don't matter, \psi = don't matter$$


In [68]:
# Gate and noise parameters
para_p = 0.001                  # depolarizing probability
para_T1 = 50e-6                 # relaxation time (seconds)
para_T2 = 70e-6                 # dephasing time (seconds)
para_tg = 35e-9                 # gate time (seconds)

def make_NI():
    para_theta = 2*np.pi            # pi/2 rotation
    para_phi = 0                    # doesn't matter 
    para_psi = np.pi/2              # doesn't matter
    NI = (-1) * gate_construction(para_theta, para_phi,para_psi, para_p, para_T1, para_T2, para_tg)
    return NI
def make_NX():
    para_theta = np.pi          # pi/2 rotation
    para_phi = 0                    # axis of rotation
    para_psi = np.pi/2
    NX = 1j * gate_construction(para_theta, para_phi, para_psi, para_p, para_T1, para_T2, para_tg)
    return NX
def make_NY():
    para_theta = np.pi         # pi/2 rotation
    para_phi = np.pi/2                 # axis of rotation
    para_psi = np.pi/2
    NY = 1j * gate_construction(para_theta, para_phi, para_psi, para_p, para_T1, para_T2, para_tg)
    return NY
def make_NZ():
    para_theta = np.pi         # pi/2 rotation
    para_phi = np.pi                 # axis of rotation
    para_psi = np.pi
    NZ = (-1j) * gate_construction(para_theta, para_phi, para_psi, para_p, para_T1, para_T2, para_tg)
    return NZ
def make_NH():
    NH = 1/sqrt(2) * (make_NX() + make_NZ())
    return NH


print(make_NI())
print(make_NX())
print(make_NY())
print(make_NZ())
print(make_NH())


[[ 1.01126931-0.00075418j  0.00727022+0.01859949j]
 [-0.0072706 +0.02791993j  0.98794385+0.00080379j]]
[[ 0.00339524+0.04585992j  0.98812763+0.00112713j]
 [ 1.00939576-0.00114221j -0.00339476+0.04852617j]]
[[ 0.03129668-0.02412557j -0.00484684-0.99917682j]
 [-0.00461738+0.99917177j -0.02208064-0.02434621j]]
[[ 0.99441795+0.03627102j  0.03086613-0.02683443j]
 [ 0.0308559 +0.00264514j -1.00292544+0.03583079j]]
[[ 0.70493033+0.01414068j  0.68953875+0.00345247j]
 [ 0.70933938+0.0084576j  -0.70639787+0.02242674j]]


## Two Gate Implementation 

### Non-Unitary Gate Simulation

In [81]:

from MsCherilynSimulator import MsCherilynSimulator, EfficientCircuit


sim = MsCherilynSimulator(
    make_NX=make_NX,
    make_NY=make_NY,
    make_NZ=make_NZ,
    make_NI=make_NI,
    make_NH=make_NH,
)



In [82]:

nqubits = 2
psi0 = np.zeros(2**nqubits, dtype=complex)
psi0[0] = 1.0  # |00>


In [ ]:
# Define state initialization (on qubits 1 and 2)
A, B, C, D = 1/2, 0, 0, 1/2
norm = sqrt(A**2 + B**2 + C**2 + D**2)
psi0 = np.zeros(2**4, dtype=complex)
psi0[1*4 + 2] = A / norm  # |01> for qubits 1,2
psi0[3*4 + 2] = D / norm  # |11> for qubits 1,2 (manually constructed)

# Create the circuit data list
cycles = 6
circuit_data = []

for i in range(cycles):
    circuit_data.extend([
        ("h", 0),  # NH on q0 (replaced by noisy H)
        ("id", 1),
        ("id", 2),
        ("id", 3),
        ("cx", (0, 1)),
        ("cx", (0, 2)),
        ("cx", (1, 3)),
        ("cx", (2, 3)),
        ("h", 0),  # NH on q0
        ("id", 1),
        ("id", 2),
        ("id", 3),
        ("measure_x", 0),
        ("measure_z", 3)
    ])
    if i != cycles - 1:
        circuit_data.extend([
            ("reset", 0),
            ("reset", 3)
        ])


In [86]:
circuit_data = [
    ("h", 0),     # Apply noisy H to qubit 0
    ("x", 1),     # Apply noisy X to qubit 1
]
probs = sim.run(circuit_data, nqubit=nqubits, psi0=psi0)


In [87]:
for state, prob in probs.items():
    print(f"|{state}⟩: {prob:.4f}")



|00⟩: 0.0011
|01⟩: 0.5445
|10⟩: 0.0010
|11⟩: 0.5119


In [ ]:


# Create simulator
mr_sim = MrAndersonSimulator()

# ---------------------------
# Part 1: Noisy circuit block
# ---------------------------
qr = QuantumRegister(2)
noisy_qc = QuantumCircuit(qr)
noisy_qc.h(0)
noisy_qc.cx(0, 1)

# Run with noise
noisy_state = mr_sim.run(noisy_qc, noise=True)

# ---------------------------
# Part 2: Clean circuit block
# ---------------------------
clean_qc = QuantumCircuit(qr)
clean_qc.initialize(noisy_state.data, qr)  # Start from previous state
clean_qc.z(1)
clean_qc.h(0)

# Run without noise
clean_state = mr_sim.run(clean_qc, noise=False)

print("Final state after noisy + clean:")
print(clean_state)

### 3. Create a quantum circuit with 4 qubits and 4 classical bits

In [72]:
def create_state(A, B, C, D):
    return np.array([A, B, C, D])

# Create a quantum circuit with 4 qubits and 4 classical bits
from numpy import sqrt

cycles = 6
qc = QuantumCircuit(4, cycles*2)
# Define the arbitrary entangled 2-qubit state
A = 1/2
B = 0
C = 0
D = 1/2

# Normalize the state vector
norm = sqrt(A**2 + B**2 + C**2 + D**2)
state_vector = create_state(A, B, C, D) / norm

# Reset q1 and q2 (qubit indices 1 and 2)
qc.reset(1)
qc.reset(2)
qc.reset(0)
qc.reset(3)

qc.initialize(state_vector, [1, 2])



for i in range(cycles):
    qc.barrier(label=f"Cycle {i}")
    NH_1 = UnitaryGate(make_NH(), label="NH")  # wrap it
    qc.append(NH_1, [0])
    NI_1_1 = make_NI()
    qc.append(NI_1_1, [1])
    NI_1_2 = make_NI()
    qc.append(NI_1_2, [2])
    NI_1_3 = make_NI()
    qc.append(NI_1_3, [3])
         
    # Apply CNOT gates as per the circuit diagram
    qc.cx(0, 1)  # CNOT from qubit 0 to qubit 1
    qc.cx(0, 2)  # CNOT from qubit 0 to qubit 2
    qc.cx(1, 3)  # CNOT from qubit 2 to qubit 1
    qc.cx(2, 3)  # CNOT from qubit 2 to qubit 3
    qc.barrier()
    # Apply Identity (I) on qubit 3 (corresponding to |lg>) - no explicit gate needed

    # Apply Hadamard gate on qubit 0 before measurement in the X basis
    NH_2 = make_NH()
    qc.append(NH_2, [0])
    NI_2_1 = make_NI()
    qc.append(NI_2_1, [1])
    NI_2_2 = make_NI()
    qc.append(NI_2_2, [2])
    NI_2_3 = make_NI()
    qc.append(NI_2_3, [3])

    # Measure qubits in the X and Z bases
    qc.measure(0, 2*i)  # X_a measurement
    qc.measure(3, 2*i+1)  # Z_b measurement

    
    if (i != cycles-1):
        # Reset qubits q0 and q3 to |0⟩ state
        qc.barrier()
        qc.reset(0)  # Reset q0
        qc.reset(3)  # Reset q3
    


# Draw the circuit
qc.draw("mpl")

ValueError: Input matrix is not unitary.